# CityBikes Rest API

This section asks you to fetch JSON data from the [CityBikes REST API](https://api.citybik.es/v2/) to answer some questions about bike share programs.

Please be mindful not to send requests to the server more often than necessary. In particular, make sure you separate code that requests data from the server into a separate cell so that you do not have to repeatedly request data from the server. You will lose points if your requests are not separated into their own cell.

In [2]:
import requests
response = requests.get("http://api.citybik.es/v2/networks")

In [3]:
import json
import pandas as pd
from pandas.io.json import json_normalize

#print(response.text[:1000])
data_bikes = response.json()
bikes = data_bikes["networks"]
bikes[0]
df_bikes = json_normalize(bikes)
df_bikes.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN
1,[Gobike A/S],/v2/networks/bycyklen,bycyklen,Bycyklen,Copenhagen,DK,55.673582,12.564984,NaN,NaN,NaN,NaN
2,[Gobike A/S],/v2/networks/nu-connect,nu-connect,Nu-Connect,Utrecht,NL,52.117000,5.067000,NaN,NaN,NaN,NaN
3,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343,NaN,NaN,NaN,NaN
4,[Gobike A/S],/v2/networks/bysykkelen,bysykkelen,Bysykkelen,Stavanger,NO,58.969975,5.733107,NaN,NaN,NaN,NaN


# Question 1

Find all cities with bike share programs in the United States (country code "US"), along with their network ID (for example, "ford-gobike"). How many cities in the U.S. have bike share programs (at least that are in this API)?

In [4]:
us = df_bikes[df_bikes["location.country"] == "US"]
len(us["id"].unique())

64

# Question 2

Construct a `DataFrame` containing data about all bike stations in all networks in the United States. Save this `DataFrame` to disk using `.to_csv()`. (You will need it in Part B of this lab.)

Use this `DataFrame` to determine the total number of bicycles in bike share programs across the United States? You may assume that the number of bikes at a station is the number of empty spaces, plus the number of available bikes.

In [22]:
import time

df_us = []
df_stations = []
for x in range(0,64):
    network_id = us["id"].iloc[x]
    response = requests.get("http://api.citybik.es/v2/networks/%s" % network_id)
    data_bikes = response.json()
    bikes = data_bikes["network"]
    df_stations.append(json_normalize(bikes, "stations", meta="name", meta_prefix="network."))
    df_us.append(json_normalize(bikes))
    time.sleep(0.499)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


In [23]:
bikes

{'company': ['Motivate LLC'],
 'gbfs_href': 'https://gbfs.baywheels.com/gbfs/gbfs.json',
 'href': '/v2/networks/bay-wheels',
 'id': 'bay-wheels',
 'location': {'city': 'San Francisco Bay Area, CA',
  'country': 'US',
  'latitude': 37.7141454,
  'longitude': -122.25},
 'name': 'Bay Wheels',
 'stations': [{'empty_slots': 10,
   'extra': {'address': None,
    'ebikes': 0,
    'has_ebikes': True,
    'last_updated': 1645574501,
    'renting': 1,
    'returning': 1,
    'uid': '340'},
   'free_bikes': 5,
   'id': 'd0e8f4f1834b7b33a3faf8882f567ab8',
   'latitude': 37.849735,
   'longitude': -122.270582,
   'name': 'Harmon St at Adeline St',
   'timestamp': '2022-02-23T03:54:22.093000Z'},
  {'empty_slots': 5,
   'extra': {'address': None,
    'ebikes': 0,
    'has_ebikes': True,
    'last_updated': 1645532780,
    'renting': 1,
    'returning': 1,
    'uid': '341'},
   'free_bikes': 6,
   'id': '983514094dd808b1604da2dcfc2d09af',
   'latitude': 37.33618830029063,
   'longitude': -121.88927650

In [24]:
df_us_bikes = pd.concat(df_us)
df_us_bikes

,company,gbfs_href,href,id,name,stations,location.city,location.country,location.latitude,location.longitude
0,"[PBSC, Alta Bicycle Share, Inc]",https://asp.publicbikesystem.net/ube/gbfs/v1/g...,/v2/networks/we-cycle,we-cycle,WE-cycle,"[{'empty_slots': 5, 'extra': {'address': 'Basa...","Aspen, CO",US,39.194951,-106.837002
0,"[Clean Energy Coalition, BCycle, LLC]",https://gbfs.bcycle.com/bcycle_arborbike/gbfs....,/v2/networks/arborbike,arborbike,ArborBike,"[{'empty_slots': 7, 'extra': {'address': '2375...","Ann Arbor, MI",US,42.278530,-83.745360
0,"[BCycle, LLC]",https://gbfs.bcycle.com/bcycle_austin/gbfs.json,/v2/networks/austin,austin,Austin B-cycle,"[{'empty_slots': 9, 'extra': {'address': '672 ...","Austin, TX",US,30.264080,-97.743550
0,"[Motivate International, Inc, PBSC]",https://chat.publicbikesystem.net/ube/gbfs/v1/,/v2/networks/bike-chattanooga,bike-chattanooga,Bike Chattanooga,"[{'empty_slots': 10, 'extra': {'address': 'Mar...","Chattanooga, TN",US,35.045630,-85.309680
0,"[Portland Bureau of Transportation (PBOT), Mot...",http://biketownpdx.socialbicycles.com/opendata...,/v2/networks/biketown,biketown,BIKETOWN,"[{'empty_slots': 6, 'extra': {'address': '5008...","Portland, OR",US,45.521754,-122.681079
...,...,...,...,...,...,...,...,...,...,...
0,[Nextbike GmbH],NaN,/v2/networks/jerseybike-guttenberg,jerseybike-guttenberg,JerseyBike,"[{'empty_slots': None, 'extra': {'bike_uids': ...",Guttenberg,US,40.792000,-74.003500
0,[Nextbike GmbH],NaN,/v2/networks/jerseybike-west-new-york,jerseybike-west-new-york,JerseyBike,"[{'empty_slots': 0, 'extra': {'bike_uids': ['7...",West New York,US,40.788200,-74.014200
0,[Nextbike GmbH],NaN,/v2/networks/jerseybike-north-bergen,jerseybike-north-bergen,JerseyBike,"[{'empty_slots': 3, 'extra': {'bike_uids': ['7...",North Bergen,US,40.804500,-74.013500
0,[Nextbike GmbH],NaN,/v2/networks/jerseybike-liberty-state-park,jerseybike-liberty-state-park,JerseyBike,"[{'empty_slots': None, 'extra': {'bike_uids': ...",Liberty State Park,US,40.705100,-74.052000


In [25]:
stations = pd.concat(df_stations)
stations["empty_slots"].sum() + stations["free_bikes"].sum() 


130985.0

In [27]:
stations.to_csv('us_bike_stations.csv', index=False)

In [10]:
import numpy as np
stations['index'] = np.arange(len(stations))
stations = stations.set_index('index')
stations.head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.last_updated,extra.renting,extra.returning,extra.uid,extra.ebikes,extra.has_ebikes,extra.bike_uids,extra.number,extra.slots
index,,,,,,,,,,,,,,,,,
0,5,6,041409c38a1cc57e8a0c49b5ef8458fe,39.394798,-107.095600,Sopris View,2021-11-02T14:43:14.673000Z,Basalt,1.635864e+09,1.0,1.0,1421,NaN,NaN,NaN,NaN,NaN
1,6,3,de99fa376a6ff38db656affe33d40e52,39.380066,-107.081968,Evans Rd,2021-11-02T14:43:14.732000Z,Basalt,1.635864e+09,1.0,1.0,1482,NaN,NaN,NaN,NaN,NaN
2,6,1,d0a1eda25363664537ff8efb3840a740,39.396610,-107.092131,English in Action,2021-11-02T14:43:14.729000Z,Basalt,1.635864e+09,1.0,1.0,1473,NaN,NaN,NaN,NaN,NaN
3,6,1,d4e64b89339be8aafa37203a16957568,39.399257,-107.090126,JW DR | El Jebel Road,2021-11-02T14:43:14.731000Z,Basalt,1.635864e+09,1.0,1.0,1474,NaN,NaN,NaN,NaN,NaN
4,7,0,ee081bd0c783181e3959e907c77d7dfa,39.391593,-107.087100,East Valley Road,2021-11-02T14:43:14.734000Z,Basalt,1.635864e+09,1.0,1.0,1436,NaN,NaN,NaN,NaN,NaN


# Question 3

You have just finished touring Coit Tower in San Francisco, which is located at latitude 37.802747 and longitude -122.405861. Using your `DataFrame` from Question 2, find the nearest bike station with an available bike, based on taxicab distance (a.k.a. Manhattan distance).

_Hint:_ You can check your answer using Google Maps!

In [11]:
from sklearn.metrics.pairwise import manhattan_distances
df_latlong = stations[["latitude", "longitude"]]
d = {'latitude': [37.802747], 'longitude': [-122.405861]}
df_coit = pd.DataFrame(data = d)
arr = manhattan_distances(df_latlong, df_coit)
nearest = arr.argmin().astype(int)
stations["name"][nearest]

'The Embarcadero at Sansome St'

Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6A - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.